In [ ]:
#from: https://github.com/iShkiper/DSP_24.M20_21/blob/main/%D0%9A%D0%BE%D0%B4/1.1.0%20DFT_1D_on.ipynb

In [9]:
from scipy.fftpack import fft, ifft, fftshift
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import get_window


fs, sig = wavfile.read('source\signals\phoneNumber'+str(0)+'.wav')

In [16]:
# DTMF частоты
ROW_FREQS = [697, 770, 852, 941]
COL_FREQS = [1209, 1336, 1477]
DTMF_MAP = {
    (697, 1209): '1', (697, 1336): '2', (697, 1477): '3',
    (770, 1209): '4', (770, 1336): '5', (770, 1477): '6',
    (852, 1209): '7', (852, 1336): '8', (852, 1477): '9',
    (941, 1209): '*', (941, 1336): '0', (941, 1477): '#'
}


In [14]:
def freq_from_fft(signal, rate, freqs):  # детект частот
    N = len(signal)
    windowed = signal * get_window('hann', N)
    spectrum = np.abs(np.fft.rfft(windowed))
    freqs_bins = np.fft.rfftfreq(N, 1 / rate)
    
    detected = {}
    for target in freqs:
        idx = np.argmin(np.abs(freqs_bins - target))
        detected[target] = spectrum[idx]
    return detected

In [22]:
freq_from_fft(sig,fs,ROW_FREQS)

{697: np.float64(286.80949291999747),
 770: np.float64(5.430858378844015),
 852: np.float64(873.9289535699147),
 941: np.float64(17.317725975926216)}

In [165]:
def decode_with_bins( sample_rate,sig, binsize=300, split=4, threshold_repeat=20):
    step = binsize // split
    prev_value = None
    prev_count = 0
    digits = []
    
   
    
    for start in range(0, len(sig)-binsize, step):
        chunk = sig[start:start+binsize]
        
        
        for i in range(0, binsize, binsize//split):
            sub = chunk[i:i+binsize//split]
            low_mags = freq_from_fft(sub, fs, ROW_FREQS)
            high_mags = freq_from_fft(sub, fs, COL_FREQS)
            low = max(low_mags, key=low_mags.get)
            high = max(high_mags, key=high_mags.get)
            threshold = 1
            
            
            digit = DTMF_MAP.get((low, high))
                
            if digit == prev_value:
                prev_count += 1
                if prev_count == threshold_repeat:
                    digits.append(digit)
            else:        
                prev_value = digit   # maybe should add new number here
                prev_count = 1
    return ''.join(digits)

In [101]:
decode_with_bins(fs,sig)

'891174501668'

In [178]:
for i in range(10):
    fs, sig = wavfile.read('source\signals\phoneNumber'+str(i)+'.wav')
    
   #### декодировать сигнал sig ####
    number = decode_with_bins(fs,sig,binsize=306,split=4,threshold_repeat=9)
    print(number+'  len:'+str(len(number))) # the great parameters hacking

89219897977  len:11
89212371218  len:11
89213902833  len:11
89022211780  len:11
89022731180  len:11
89021229912  len:11
89112079312  len:11
89212893227  len:11
89118319033  len:11
89117120138  len:11
